In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
# %matplotlib notebook

Populating the interactive namespace from numpy and matplotlib


## 0. Import module

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
import os
from torchsummary import summary
from torchvision import transforms                                                                                                                                        
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 600
from PIL import Image

import sys
sys.path.append("../src/")
import model
from datasets import imagenet
import config
from aux.utils import obtain_features_map, load_imgs, zscore, extract_valid
from aux.visualization import visualize_features_map

## 1.Hyperparameter

In [4]:
resume = "037-0"
model_dir = "../saved/models"
generated_dir = "../saved/generated/"
backbone = "vgg16"

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 2.Load modal from dict

In [5]:
# model
net = model.Network(backbone=backbone, num_classes=1000)
net.to(device)

# resume from model
resume_exp = resume.split("-")[0]
resume_epoch = resume.split("-")[1]
print("Resume from model from exp: {} at epoch {}".format(resume_exp, resume_epoch))
resume_path = os.path.join(model_dir, str(resume_exp), str(resume_epoch))
ckpt = torch.load(resume_path, map_location=device)
net.load_state_dict(ckpt) 

> Use original fc
Resume from model from exp: 037 at epoch 0


<All keys matched successfully>

## 3.1 Load Original Data 

In [23]:
# Load data
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]
reverse_mean = [-0.485, -0.456, -0.406]
reverse_std = [1/0.229, 1/0.224, 1/0.225]
train_transform = None
train_transform = transforms.Compose([                                                                                                                                
    transforms.Resize((224, 224), interpolation=Image.BILINEAR),                                                                                              
    # transforms.RandomCrop(input_size),                                                                                                                              
    # transforms.RandomHorizontalFlip(),                                                                                                                              
    # transforms.ColorJitter(brightness=0.4, saturation=0.4, hue=0.4),                                                                                                
    transforms.ToTensor(),                                                                                                                                            
    transforms.Normalize(mean, std)                                                                                                                                   
       ])         
trainset = imagenet.ImageNet(root="/media/lincolnzjx/HardDisk/Datasets/", is_train=True, transform=train_transform)

trainset.set_data([950], 30)
#image, label, imgs_path = trainset
imgs_path = []                                                                                                                                                            
images = []
labels = []
for img, label, img_path in trainset:                                                                                                                                     
    images.append(img.unsqueeze(0))                                                                                                                                       
    labels.append(label)                                                                                                                                                  
    imgs_path.append(img_path)  

Len of new dataset is :30


In [24]:
imgs_path

['/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10061.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10068.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10110.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10134.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10162.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10166.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10178.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10179.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10184.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10192.JPEG',
 '/media/lincolnzjx/HardDisk/Datasets/ilsvrc2012/train/n07747607/n07747607_10204.JPEG',
 '/media/lincolnzjx/HardDisk/Dat

## 3.2 Load optimized data from given path

In [25]:
save_dir = "../saved/pack/"
################### Hyper-Parameter #######################
exp = "033002"
epoch = "99900"
##########################################################
ab_path = os.path.join(save_dir, exp, epoch)

In [26]:
# Load image
optimized_data, valid_imgs_path, valid_imgs_index = load_imgs(ab_path, imgs_path, non_exists_ok=True)
valid_imgs, valid_labels = extract_valid(images, labels, valid_imgs_index)
optimized_data = zscore(optimized_data, mean, std)

Load from ../saved/pack/033002/99900/n07747607_10061.JPEG
Load from ../saved/pack/033002/99900/n07747607_10068.JPEG
Load from ../saved/pack/033002/99900/n07747607_10110.JPEG
Load from ../saved/pack/033002/99900/n07747607_10134.JPEG
Load from ../saved/pack/033002/99900/n07747607_10162.JPEG
Load from ../saved/pack/033002/99900/n07747607_10166.JPEG
Load from ../saved/pack/033002/99900/n07747607_10178.JPEG
Load from ../saved/pack/033002/99900/n07747607_10179.JPEG
Load from ../saved/pack/033002/99900/n07747607_10184.JPEG
Load from ../saved/pack/033002/99900/n07747607_10192.JPEG
Load from ../saved/pack/033002/99900/n07747607_10204.JPEG
Load from ../saved/pack/033002/99900/n07747607_10205.JPEG
Load from ../saved/pack/033002/99900/n07747607_1021.JPEG
Load from ../saved/pack/033002/99900/n07747607_10213.JPEG
Load from ../saved/pack/033002/99900/n07747607_10217.JPEG
Load from ../saved/pack/033002/99900/n07747607_10232.JPEG
Load from ../saved/pack/033002/99900/n07747607_10258.JPEG
Load from ../sa

In [27]:
index2image = {index: item.split("/")[-1].split(".")[0] for index, item in enumerate(valid_imgs_path)}
index2image

{0: 'n07747607_10061',
 1: 'n07747607_10068',
 2: 'n07747607_10110',
 3: 'n07747607_10134',
 4: 'n07747607_10162',
 5: 'n07747607_10166',
 6: 'n07747607_10178',
 7: 'n07747607_10179',
 8: 'n07747607_10184',
 9: 'n07747607_10192',
 10: 'n07747607_10204',
 11: 'n07747607_10205',
 12: 'n07747607_1021',
 13: 'n07747607_10213',
 14: 'n07747607_10217',
 15: 'n07747607_10232',
 16: 'n07747607_10258',
 17: 'n07747607_10290',
 18: 'n07747607_10305',
 19: 'n07747607_10345',
 20: 'n07747607_10358',
 21: 'n07747607_10379',
 22: 'n07747607_10420',
 23: 'n07747607_10425',
 24: 'n07747607_10436',
 25: 'n07747607_10465',
 26: 'n07747607_10482',
 27: 'n07747607_10485',
 28: 'n07747607_10491',
 29: 'n07747607_10545'}

In [28]:
save_dict = {
    "save_dir": "../saved/pack/"+ exp + "/DiffMap/",
    "index2image": index2image, 
    "save_name": "{}.pdf"
}

try: 
    os.mkdir(save_dict["save_dir"])
except FileExistsError:
    print("Directory has been created {}".format(save_dict["save_dir"]))

Directory has been created ../saved/pack/033002/DiffMap/


In [31]:
# Move to device
opt_image = torch.from_numpy(optimized_data).to(device)
original_image = torch.cat(valid_imgs, dim=0).to(device)

## 4. Obtain feature from conv layers.

In [32]:
# hyper parameter
conv_output_indexes = [1, 3, 6, 8, 11, 13, 15, 18, 20, 22, 25, 27, 29]
conv_output_indexes_dict = dict(zip(conv_output_indexes, range(len(conv_output_indexes))))
print(conv_output_indexes_dict)

{1: 0, 3: 1, 6: 2, 8: 3, 11: 4, 13: 5, 15: 6, 18: 7, 20: 8, 22: 9, 25: 10, 27: 11, 29: 12}


In [37]:
origin_layer_output, _ = obtain_features_map(original_image, net.model.features, layer_output_indexes=conv_output_indexes)
opt_layer_output, _ = obtain_features_map(opt_image, net.model.features, layer_output_indexes=conv_output_indexes)

0 Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) torch.Size([30, 64, 224, 224])
1 ReLU(inplace=True) torch.Size([30, 64, 224, 224])
Index:1, ReLU(inplace=True)
0.0 17.085962
2 Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) torch.Size([30, 64, 224, 224])
3 ReLU(inplace=True) torch.Size([30, 64, 224, 224])
Index:3, ReLU(inplace=True)
0.0 33.326855
4 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) torch.Size([30, 64, 112, 112])
5 Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) torch.Size([30, 128, 112, 112])
6 ReLU(inplace=True) torch.Size([30, 128, 112, 112])
Index:6, ReLU(inplace=True)
0.0 53.61295
7 Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)) torch.Size([30, 128, 112, 112])
8 ReLU(inplace=True) torch.Size([30, 128, 112, 112])
Index:8, ReLU(inplace=True)
0.0 74.994286
9 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) torch.Size([30, 128, 56, 56])
10 Conv2

## 5. DiffMap

13